# Processing results using BigQuery #

We start by importing all the requisite packages from BayesCMD etc. as well as ones required to plot and read data from big query.

In [1]:
import os
import argparse
from pathlib import Path
import json
import matplotlib.pyplot as plt
from distutils import dir_util
from pprint import pprint
import pickle

# BayesCMD packages 
from bayescmd.results_handling import kde_plot
from bayescmd.results_handling import scatter_dist_plot
from bayescmd.results_handling import data_import
from bayescmd.results_handling import plot_repeated_outputs
from bayescmd.results_handling import histogram_plot
from bayescmd.results_handling import data_merge_by_batch
from bayescmd.abc import import_actual_data
from bayescmd.abc import priors_creator

# Google BigQuery
from google.cloud import bigquery
%load_ext google.cloud.bigquery

STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD
STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD
STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD


In [2]:
# Explicitly use service account credentials by specifying the private
# key file. All clients in google-cloud-python have this helper.
client = bigquery.Client.from_service_account_json(
    "../../gcloud/hypothermia-auth.json"
)

In [3]:
def generate_histogram_query(project, dataset, model, n_bins, distance):
    histogram_query = """
    SELECT
      MIN(data.{distance}) AS min,
      MAX(data.{distance}) AS max,
      COUNT(data.{distance}) AS num,
      INTEGER((data.{distance}-value.min)/(value.max-value.min)*{n_bins}) AS group_
    FROM
      [{project}:{dataset}.{model}] data
    CROSS JOIN (
      SELECT
        MAX({distance}) AS max,
        MIN({distance}) AS min
      FROM
        [{project}:{dataset}.{model}]) value
    GROUP BY
      group_
    ORDER BY
      group_
    """.format(dataset=dataset, model=model, n_bins=n_bins, distance=distance, project=project)
    return histogram_query

In [4]:
def generate_posterior_query(project, dataset, model, distance, parameters, limit=50000):
    unpacked_params = ",\n".join(parameters)
    histogram_query = """
SELECT
    {unpacked_params},
    {distance},
    idx
FROM
  `{project}.{dataset}.{model}`
ORDER BY
  {distance} ASC
LIMIT
  {limit}
    """.format(project=project, dataset=dataset, model=model, unpacked_params=unpacked_params,distance=distance, limit=limit)
    return histogram_query

In [5]:
def load_configuration(model_version, dataset, verbose=False):
    current_file = Path(os.path.abspath(''))
    config_file = os.path.join(current_file.parents[2],
                              'config_files',
                               'abc',
                               'bp_hypothermia_{}'.format(model_version),
                               'bp_hypothermia_{}_config.json'.format(model_version)
                              )

    with open(config_file, 'r') as conf_f:
        conf = json.load(conf_f)

    params = conf['priors']

    input_path = os.path.join(current_file.parents[2],
                              'data',
                              'clean_hypothermia',
                              '{}_filtered_formatted.csv'.format(dataset.upper()))

    d0 = import_actual_data(input_path)

    targets = conf['targets']
    model_name = conf['model_name']
    inputs = conf['inputs']

    config = {
        "model_name": model_name,
        "targets": targets,
        "inputs": inputs,
        "parameters": params,
        "input_path": input_path,
        "zero_flag": conf['zero_flag'],
    }
    
    if verbose:
        pprint(config)
        
    return config, d0

In [6]:
configuration = {}
combinations = [('1', 'LWP475'), ('1_1', 'LWP479'),
                ('2', 'LWP475'), ('2_1', 'LWP479'),
                ('4', 'LWP475'), ('4_1', 'LWP479')]


for combo in combinations:
    
    model_number = combo[0]
    DATASET = combo[1]

    model_name = 'bph{}'.format(model_number)
    configuration[model_name] = {}

    configuration[model_name][DATASET] = {}
    config, d0 = load_configuration(model_number, DATASET)
    configuration[model_name][DATASET]['bayescmd_config'] = config
    configuration[model_name][DATASET]['original_data']= d0

    configuration[model_name][DATASET]['histogram_query'] = generate_histogram_query('hypothermia-bayescmd', 
                                                                                        DATASET, 
                                                                                        model_name, 
                                                                                        100, 
                                                                                        'NRMSE')

    configuration[model_name][DATASET]['posterior_query'] = generate_posterior_query('hypothermia-bayescmd', 
                                                                                        DATASET, 
                                                                                        model_name, 
                                                                                        'NRMSE', 
                                                                                        list(configuration[model_name][DATASET]['bayescmd_config']['parameters'].keys()))


In [7]:
import string
labels = {"t": "Time (sec)",
              "SaO2sup": "SaO2 (%)",
              "P_a": "ABP (mmHg)",
              "temp": "Temperature ($^{\circ}$C)",
               "TOI": "TOI (%)",
              "HbO2": "$\Delta$HbO$_2$ $(\mu M)$",
              "HHb": "$\Delta$HHb $(\mu M)$",
              "CCO": "$\Delta$CCO $(\mu M)$"
             }
LIM=50000
combinations = [('bph1', 'LWP475'), ('bph1_1', 'LWP479'),
                ('bph2', 'LWP475'), ('bph2_1', 'LWP479'),
                ('bph4', 'LWP475'), ('bph4_1', 'LWP479')]

signals = ['CCO', 'HbO2', 'HHb']
# for model_name in ['bph2_1']:
#     for DATASET in ['LWP479']:#, 'LWP479' ,'LWP481' ,'LWP484']:
for fig_num, combo in enumerate(combinations):
    DATASET=combo[1]
    model_name=combo[0]
    print("Working on {} - {}".format(model_name, DATASET))
    # Set config and create figure path
    config = configuration[model_name][DATASET]['bayescmd_config']
    figPath = "/home/buck06191/Dropbox/phd/hypothermia/ABC/Figures/{}/{}/{}".format(model_name, DATASET, 'NRMSE')
    dir_util.mkpath(figPath)

    # Get posterior
    print("\tRunning SQL query")
    df_post = client.query(configuration[model_name][DATASET]['posterior_query']).to_dataframe()

    # Plot posterior 
    print("\tPlotting posterior")
#     g = kde_plot(df_post, config['parameters'], limit=LIM, n_ticks=4, d="NRMSE", median_file=os.path.join(figPath, "medians.txt"))
    #g.fig._suptitle.set_text(g.fig._suptitle.get_text() + 'for {} - {}'.format(DATASET, model_name))
#     with open(os.path.join(figPath, 'posterior_{}_{}.pickle'.format(model_name, DATASET)), 'wb') as f: 
#         pickle.dump(g, f)
#     g.fig.savefig(
#         os.path.join(figPath, 'posterior_{}_{}.png'
#                      .format(model_name, DATASET)),
#         bbox_inches='tight', dpi=250)

    # Plot posterior predictive
    config["offset"] = {}
    print("\tPlotting Posterior Predictive")
    fig, axes, lgd = plot_repeated_outputs(df_post, n_repeats=5000, limit=LIM,
                                    distance='NRMSE', **config)
    for ix, ax in enumerate(axes.flatten()):
        ax.set_ylabel(labels[signals[ix]])
        old_title = ax.get_title().split(':')
        new_title = ":".join([labels[signals[ix]].split()[0], old_title[1]])
        ax.set_title(new_title, size=11)
    axes.flatten()[-1].set_xlabel(labels['t'])
    fig.suptitle(string.ascii_lowercase[fig_num]+")", ha='left', x=-0.02, y=0.925)
    #fig.set_size_inches(18.5, 12.5)
    with open(os.path.join(figPath, 'posterior_predictive_{}_{}.pickle'.format(model_name, DATASET)), 'wb') as f: 
        pickle.dump(fig, f)
    fig.savefig(
        os.path.join(figPath, 'posterior_predictive_{}_{}.png'
                     .format(model_name, DATASET)),
        bbox_inches='tight', bbox_extra_artists=(lgd,), dpi=250)
        #plt.close('all')

Working on bph1 - LWP475
	Running SQL query
	Plotting posterior
	Plotting Posterior Predictive
Sample 0, idx:10410
Sample 1, idx:41221
Sample 2, idx:2946
Sample 3, idx:14477
Sample 4, idx:48220
Sample 5, idx:41007
Sample 6, idx:34756
Sample 7, idx:17280
Sample 8, idx:12857
Sample 9, idx:23614
Sample 10, idx:41764
Sample 11, idx:35098
Sample 12, idx:3131
Sample 13, idx:18747
Sample 14, idx:32577
Sample 15, idx:18600
Sample 16, idx:2108
Sample 17, idx:16341
Sample 18, idx:33837
Sample 19, idx:3071
Sample 20, idx:4581
Sample 21, idx:45931
Sample 22, idx:20399
Sample 23, idx:28994
Sample 24, idx:10765
Sample 25, idx:48776
Sample 26, idx:42767
Sample 27, idx:40372
Sample 28, idx:8908
Sample 29, idx:31762
Sample 30, idx:39571
Sample 31, idx:29161
Sample 32, idx:27524
Sample 33, idx:25560
Sample 34, idx:44527
Sample 35, idx:16225
Sample 36, idx:11892
Sample 37, idx:9650
Sample 38, idx:2784
Sample 39, idx:38455
Sample 40, idx:28536
Sample 41, idx:33775
Sample 42, idx:24274
Sample 43, idx:650
S

KeyboardInterrupt: 

In [ ]:
# fig.subplots_adjust(bottom=0.2)
# for ax in fig.axes:
#     ax.set_ylabel("$\Delta$" + ax.get_ylabel()+" ($\mu$M)")
# with open(os.path.join(figPath, 'posterior_predictive_{}_{}.pickle'.format(model_name, DATASET)), 'wb') as f: 
#             pickle.dump(fig, f)
# fig.savefig(
#     os.path.join(figPath, 'posterior_predictive_{}_{}.png'
#                  .format(model_name, DATASET)),
#     bbox_inches='tight', bbox_extra_artists=(lgd,), dpi=250)

In [ ]:
# configuration = {}
# for model_number in [1, 2, 4]:
#     model_name = 'bph{}'.format(model_number)
#     configuration[model_name] = {}
#     for DATASET in ['LWP475', 'LWP479' ,'LWP481' ,'LWP484']: 
#         configuration[model_name][DATASET] = {}
#         for output in ['CCO', 'HbO2', 'HHb']:
#             configuration[model_name][DATASET][output] = {}
#             config, d0 = load_configuration(model_number, DATASET)
#             configuration[model_name][DATASET][output]['bayescmd_config'] = config
#             configuration[model_name][DATASET][output]['original_data']= d0

#             configuration[model_name][DATASET][output]['histogram_query'] = generate_histogram_query('hypothermia-bayescmd', 
#                                                                                                 DATASET, 
#                                                                                                 model_name, 
#                                                                                                 100, 
#                                                                                                 '{}_NRMSE'.format(output))

#             configuration[model_name][DATASET][output]['posterior_query'] = generate_posterior_query('hypothermia-bayescmd', 
#                                                                                                 DATASET, 
#                                                                                                 model_name, 
#                                                                                                 '{}_NRMSE'.format(output), 
#                                                                                                 list(configuration[model_name][DATASET][output]['bayescmd_config']['parameters'].keys()))


In [ ]:
# %%capture individual_outputs
# LIM=5000
# for model_name in ['bph1', 'bph2', 'bph4']:
#     for DATASET in ['LWP475', 'LWP479' ,'LWP481' ,'LWP484']:
#         for output in ['CCO', 'HbO2', 'HHb']:
#             print("Working on {} - {}, {}\n".format(model_name, DATASET, output))
#             # Set config and create figure path
#             config = configuration[model_name][DATASET][output]['bayescmd_config']
#             figPath = "/home/buck06191/Dropbox/phd/hypothermia/ABC/Figures/{}/{}/{}".format(model_name, DATASET, '{}_NRMSE'.format(output))
#             dir_util.mkpath(figPath)

#             # Get posterior
#             df_post = client.query(configuration[model_name][DATASET][output]['posterior_query']).to_dataframe()

#             # Plot posterior 
#             g = kde_plot(df_post, config['parameters'], limit=LIM, n_ticks=4, d="{}_NRMSE".format(output), median_file=os.path.join(figPath, "medians.txt"))
#             g.fig._suptitle.set_text(g.fig._suptitle.get_text() + 'for {} - {}, {}'.format(DATASET, model_name,output))
#             g.fig._suptitle.set_fontsize(26)
#             with open(os.path.join(figPath, 'posterior_{}_{}_{}.pickle'.format(model_name, DATASET,output)), 'wb') as f: 
#                 pickle.dump(g, f)
#             g.fig.savefig(
#                 os.path.join(figPath, 'posterior_{}_{}_{}.png'
#                              .format(model_name, DATASET,output)),
#                 bbox_inches='tight', dpi=250)

#             # Plot posterior predictive
#             config["offset"] = {}
#             fig, ax, lgd = plot_repeated_outputs(df_post, n_repeats=100, limit=LIM,
#                                             distance='{}_NRMSE'.format(output), **config)
#             fig.suptitle('Posterior predictive distributions\nfor {} - {}, {}'.format(DATASET, model_name,output), fontsize=26)
#             fig.set_size_inches(18.5, 12.5)
#             with open(os.path.join(figPath, 'posterior_predictive_{}_{}_{}.pickle'.format(model_name, DATASET, output)), 'wb') as f: 
#                 pickle.dump(fig, f)
#             fig.savefig(
#                 os.path.join(figPath, 'posterior_predictive_{}_{}_{}.png'
#                              .format(model_name, DATASET,output)),
#                 bbox_inches='tight', bbox_extra_artists=(lgd,), dpi=250)
#             plt.close('all')

In [ ]:
string.ascii_lowercase[0]